In [9]:
import numpy as np
import pyautogui
import imutils
import cv2
import objc
from AppKit import NSEvent
import sys
import Quartz
import time
import random
import math
try: 
    import Image
except ImportError:
    from PIL import Image
import pytesseract

import mss
import mss.tools
from PIL import Image
import PIL.ImageOps  
import re
import os
from selenium import webdriver

In [10]:
# This function opens a new browser window, positions and sizes it as desired and then opens slither.io

def open_and_size_browser_window(width, height, x_pos = 0, y_pos = 0, url = 'http://www.slither.io'):

#     opens the browser window
    chrome_options = webdriver.ChromeOptions()    
    chrome_options.add_argument("--disable-infobars")
    driver = webdriver.Chrome("./chromedriver", chrome_options=chrome_options)

#     sets the size of the window
    driver.set_window_size(width, height)
    
#     repositions the window
    driver.set_window_position(x_pos, y_pos)
    
#     goes to slither.io
    driver.get(url)

    return driver

In [11]:
# this class is for controlling the mouse using python.  It has a bunch of functions, some of which I don't use here
class Mouse():
    down = [Quartz.kCGEventLeftMouseDown, Quartz.kCGEventRightMouseDown, Quartz.kCGEventOtherMouseDown]
    up = [Quartz.kCGEventLeftMouseUp, Quartz.kCGEventRightMouseUp, Quartz.kCGEventOtherMouseUp]
    [LEFT, RIGHT, OTHER] = [0, 1, 2]

    def position(self):
        point = Quartz.CGEventGetLocation( Quartz.CGEventCreate(None) )
        return point.x, point.y

    def location(self):
        loc = NSEvent.mouseLocation()
        return loc.x, Quartz.CGDisplayPixelsHigh(0) - loc.y

    def move(self, x, y):
        moveEvent = Quartz.CGEventCreateMouseEvent(None, Quartz.kCGEventMouseMoved, (x, y), 0)
        Quartz.CGEventPost(Quartz.kCGHIDEventTap, moveEvent)

    def press(self, x, y, button=1):
        event = Quartz.CGEventCreateMouseEvent(None, Mouse.down[button], (x, y), button - 1)
        Quartz.CGEventPost(Quartz.kCGHIDEventTap, event)

    def release(self, x, y, button=1):
        event = Quartz.CGEventCreateMouseEvent(None, Mouse.up[button], (x, y), button - 1)
        Quartz.CGEventPost(Quartz.kCGHIDEventTap, event)

    def click(self, button=LEFT):
        x, y = self.position()
        self.press(x, y, button)
        self.release(x, y, button)

    def click_pos(self, x, y, button=LEFT):
        self.move(x, y)
        self.click(button)

    def to_relative(self, x, y):
        curr_pos = Quartz.CGEventGetLocation( Quartz.CGEventCreate(None) )
        x += current_position.x;
        y += current_position.y;
        return [x, y]

    def move_rel(self, x, y):
        [x, y] = to_relative(x, y)
        moveEvent = Quartz.CGEventCreateMouseEvent(None, Quartz.kCGEventMouseMoved, Quartz.CGPointMake(x, y), 0)
        Quartz.CGEventPost(Quartz.kCGHIDEventTap, moveEvent)

In [12]:

# this function takes a screenshot of a particular area of the screen
# it also offers the ability to reduce the resolution by selection only every nth pixel...
# there are certainly better ways to reduce the resolution, but this is where I started
# it also lets you decide if you want to convert to grayscale.  
def screenshot(x, y, width, height
               , reduction_factor = 1
               , gray = True
              ):
    
    with mss.mss() as sct:
        # The screen part to capture
        region = {'left': x, 'top': y, 'width': width, 'height': height}

        # Grab the data
        img = sct.grab(region)

        if gray:
            result = cv2.cvtColor(np.array(img), cv2.COLOR_BGRA2GRAY)
        else:
            result = cv2.cvtColor(np.array(img), cv2.COLOR_BGRA2BGR)

        img = result[::reduction_factor, ::reduction_factor]
        return img

# this function looks at the right part of the screen for a screenshot of the score 
# then it uses OCR to convert the image of the score into a number
# it will take screenshots until it gets a valid numerical score
# the OCR is not 100% accurate.  The threshold and var values are used to tune the image to maximize the effectiveness of the OCR
def get_score(threshold = 190, var = 10):

    got_it = False
    
    while not got_it:
        img = screenshot(x= score_x, y= score_y, width = score_width, height = score_height, gray = False)

        for rows in range(0, len(img)):
            for cols in range(0, len(img[0])):
                x = img[rows][cols]
                if x[0] > threshold and x[1] > threshold and x[2] > threshold and max(x) - min(x) < var:
                    img[rows][cols] = [0, 0, 0]
                else:
                    img[rows][cols] = [255,255,255]

        score = pytesseract.image_to_string(img, config='outputbase digits')
        
        score = re.sub("[^0-9]", "", score)
        
        if score != "":            
            score = int(score)
            return score
    

# this function moves the mouse to a location on a circle with radius = radius
# The user specifies to position on the circle is radians
# This is used for steering the worm
def move_to_radians(radians, radius = 50):
    
    mouse.move(window_center_x + radius * math.cos(radians)
               , window_center_y + radius * math.sin(radians))
    
    return radians

# this function determines if the player is dead
# if the player is dead, then the word "Skin" appears where the score would normally be
def is_dead():
    img = screenshot(x= score_x, y= score_y, width = score_width, height = score_height, gray = False)
    if pytesseract.image_to_string(img) == 'Skin':
        return True
    else: 
        return False

# In order to start the game, you have to click on the start button
# if the window is not active, you need two clicks, so this does one click, waits one second, and then clicks again 
# and makes the worm move toward 0
def start_game():
    time.sleep(1)
    mouse.move(start_button_position_x, start_button_position_y)
    pyautogui.click()
    time.sleep(1)
    pyautogui.click()
    time.sleep(1)
    move_to_radians(0)
    
# this function determines the direction that the worm is currently moving based on the mouse's position
def get_direction():
    x, y = mouse.position
    
    return math.atan2(y, x)


In [17]:

# This function starts a long loop that plays the game.  I have started it out moving randomly
# The challenge is to make the program actually play the game effectively
# The worm often gets a surprising number of points by playing totally randomly
def generate_data(delete_file = False, time_gap = 1, verbose = False):
        
#     initializes the score to 10 -- which is what you start at
    score = 10
    
#     the program will automatically capture data about the screen and the direction, along with the score 1 second later.
#     there is a flag to control whether or not the file is appended to or overwritten
    if delete_file:
        if os.path.isfile('data.csv'):
            os.remove('data.csv')
    
#     outer loop to control the number of data points gathered
    for i in range(0, 50000):
        
#         if dead, reset the score and start a new game
        if is_dead():
            print('You are dead.  Starting the game over.')
            start_game()
            score = 10
            time.sleep(1)
        else:
            
#             randomly change direction
            direction = move_to_radians(random.uniform(0, math.pi*2))
            
#             capture data about what the screen looks like and convert it to a numpy array
            data = screenshot(x=window_x
                              , y=window_y
                              , width=window_width
                              , height=window_height
                              , reduction_factor = 5)
            
#             convert the image array into a vector that can be stored
            values = data.shape[0] * data.shape[1]
            data = data.reshape(values)
        
#             remember what the old score was            
            old_score = score
            
#             wait some time (defaults to 1 second) and then get the new score
            time.sleep(time_gap)
            score = get_score()
            
#             add the image data, the direction, along with the outcome information to a numpy array
            data = np.append(data, [direction, old_score, score, score - old_score])
            
#             write that row out to a file
            with open('data.csv', 'ab') as f:
                np.savetxt(f, [data], fmt='%5d', delimiter=',')

#             print out the results onto the screen if desired
            
            if verbose:
                print('Data point collected.  Direction: %0.2f. Score changed by: %i' % (direction, score-old_score))
            


In [18]:
# These are the dimensions of the window that I'm going to use
window_x = 0.0
window_y = 100 # The tool bar in chrome is 100 pixels (or so) tall

# Too high resolution and the dataset gets too big.  This is the size of the window to create
width = 600 
height = 600

# size of the game window (the playable area)
window_width = width 
window_height = height - window_y

# coordinates of the center of the game windows
window_center_x = window_width/2
window_center_y = window_y + window_height/2

#######################
# comment these out if you want to consider the whole window
# this is so that you can easily consider less data if you want to
size_of_window = 200.0
window_x = window_center_x - size_of_window/2
window_y = window_center_y - size_of_window/2
window_width = size_of_window
window_height = size_of_window
#######################

# Location of the score that is displayed on the screen
# This will change if you change the dimensions of the window that you create.  
# YOu can easily use mouse.position() to identify the new coordinates
score_x = 83
score_y = 580
score_width = 91
score_height = 25

# position of the start button on the screen with these window settings
# Note that the shape of the start button changes depending on whether it's a brand new game
# or if you've just died
start_button_position_x = window_center_x
start_button_position_y = 490

# instantiate mouse
mouse = Mouse()

# open slither.io
driver = open_and_size_browser_window(width = width, height = height)

# make some data
generate_data(delete_file = True, verbose = True)

You are dead.  Starting the game over.
Data point collected.  Direction: 4.69. Score changed by: 8
Data point collected.  Direction: 4.21. Score changed by: 1
Data point collected.  Direction: 4.26. Score changed by: 2
Data point collected.  Direction: 4.83. Score changed by: 1
Data point collected.  Direction: 0.67. Score changed by: 2
Data point collected.  Direction: 3.44. Score changed by: 7
Data point collected.  Direction: 2.09. Score changed by: 0
Data point collected.  Direction: 0.32. Score changed by: 1
Data point collected.  Direction: 0.60. Score changed by: 0
Data point collected.  Direction: 2.00. Score changed by: 1
Data point collected.  Direction: 2.06. Score changed by: 1
Data point collected.  Direction: 4.25. Score changed by: 7
Data point collected.  Direction: 5.83. Score changed by: 1
Data point collected.  Direction: 0.39. Score changed by: 0


KeyboardInterrupt: 